In [ ]:
import pandas as pd
import numpy as np
import h5py
import os
import datetime
import matplotlib.pyplot as plt
from matplotlib import ticker
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['axes.facecolor'] = 'w'

### Simulation parameters 

In [ ]:
year, month, day = '2020', '10', '1'
start_date = pd.Timestamp(year + '.' + month.zfill(2) + '.' + day.zfill(2))
tmax = '180'
daysPlot = 180

#define scenarios
strings = ['']

#define compartments
secir_dict = {0:'Susceptible', 1:'Exposed',2:'Carrier', 3:'Infected', 4:'Hospitalized',
              5:'ICU', 6:'Recovered', 7:'Death'}

#define age groups
age_groups = ['0-4 Years', '5-14 Years', '15-34 Years', '35-59 Years', '60-79 Years', '80+ Years']

### Plot parameters

In [ ]:
# define colors for age groups
plt_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
colors = {}
colors['Total'] = plt_colors[0]
for i in range(len(age_groups)):
    colors['Group' + str(i+1)] = plt_colors[i+1]
    


In [ ]:
# define population data for incidence values
base = 100000
age_group_sizes = np.array([3961376,7429883,19117865,28919134,18057318,5681135])

relative_dict = {}
for i in range(len(age_group_sizes)):
    relative_dict[age_groups[i]] = age_group_sizes[i]/base
    
relative_dict['Total'] = np.sum(age_group_sizes)/base

    

In [ ]:
plotRKI = True           # Plots RKI Data if true
plotRelative = False     # Plots incidence values if true
plotPercentiles = False  # Plots 25 and 75 percentiles if true
plotConfidence = False   # Plots 05 and 95 percentiles if true

In [ ]:
savePlot = True          # saves plot file if true
if savePlot:
    try:
        os.mkdir('Plots')
    except:
        print('Directory "Plots" already exists')

In [ ]:


opacity = 0.15           
lineWidth = 3.5
fontsize = 18
figsize = (16, 10)

#define x-ticks for plots
datelist = np.array(pd.date_range(start_date.date(), periods=daysPlot, freq='D').strftime('%m-%d').tolist())
tick_range = (np.arange(int(daysPlot / 10) + 1) * 10)
tick_range[-1] -= 1

# Plot Results

In [ ]:
def plot_results(files, comp_idx, title, key='Total'):
    fig, ax = plt.subplots(figsize=figsize)
    
    if plotRelative:
        factor = relative_dict[key]
    else:
        factor = 1
        
    X = np.arange(daysPlot)
    
    ax.plot(X, files['p50']['0'][key][:, comp_idx]/factor, label='p50',
            color=colors[key], linewidth=lineWidth)
    if plotPercentiles:    
        ax.plot(X, files['p25']['0'][key][:, comp_idx]/factor,'--', label='p25',
                color=colors[key], linewidth=lineWidth)
        ax.plot(X, files['p75']['0'][key][:, comp_idx]/factor,'--',  label='p75',
                color=colors[key], linewidth=lineWidth)
        ax.fill_between(X, files['p25']['0'][key][:, comp_idx]/factor,
                        files['p75']['0'][key][:, comp_idx]/factor,
                        color=colors[key], alpha=opacity) 
    if plotConfidence:    
        ax.plot(X, files['p05']['0'][key][:, comp_idx]/factor,'--', label='p05',
                color=colors[key], linewidth=lineWidth)
        ax.plot(X, files['p95']['0'][key][:, comp_idx]/factor,'--',  label='p95',
                color=colors[key], linewidth=lineWidth)
        ax.fill_between(X, files['p05']['0'][key][:, comp_idx]/factor, 
                        files['p95']['0'][key][:, comp_idx]/factor,
                        color=colors[key], alpha=opacity) 
        
    if plotRKI:
        ax.plot(X, files['RKI']['0'][key][:, comp_idx]/factor,'--', label='RKI',
            color='gray', linewidth=lineWidth)
        

    ax.set_title(title, fontsize=18)
    ax.set_xticks(tick_range)
    ax.set_xticklabels(datelist[tick_range], rotation=45, fontsize=fontsize)
    if plotRelative:
        ax.set_ylabel('individuals relative per 100.000', fontsize=fontsize)
    else:
        ax.set_ylabel('number of individuals', fontsize=fontsize)
    ax.legend(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    ax.grid(linestyle='dotted')
    
    formatter = ticker.ScalarFormatter(useMathText=True)
    formatter.set_scientific(True) 
    formatter.set_powerlimits((-1,1)) 
    ax.yaxis.set_major_formatter(formatter) 
    ax.yaxis.offsetText.set_fontsize(fontsize)
    
    if savePlot:
        fig.savefig('Plots/' + title + '.png')


In [ ]:
# load files
files = []

for i in range(len(strings)):
    files.append({})
    
    path = '../data/results/results_' + year + '_' + month + '_' + day + '_tmax_' + tmax + strings[i]

    files[i]['p50'] = h5py.File(path + '/results_p50/Results_sum.h5', 'r+')

    files[i]['p25'] = h5py.File(path + '/results_p25/Results_sum.h5', 'r+')
    files[i]['p75'] = h5py.File(path + '/results_p75/Results_sum.h5', 'r+')

    files[i]['p05'] = h5py.File(path + '/results_p05/Results_sum.h5', 'r+')
    files[i]['p95'] = h5py.File(path + '/results_p95/Results_sum.h5', 'r+')
        

    files[i]['RKI'] = h5py.File(path + '/Results_rki_sum.h5', 'r+')
        

### Plot total population 

In [ ]:
for scenario in range(len(strings)):
    for compart in range(len(secir_dict)):
        plot_results(files[scenario], compart, secir_dict[compart], key='Total')

### Plot group population

In [ ]:
for scenario in range(len(strings)):
    for compart in range(len(secir_dict)):
        for group in range(len(age_groups)):
            plot_results(files[scenario], compart, secir_dict[compart] + ' ' + age_groups[group],
                         key='Group' + str(group+1))